In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time

In [2]:
keys = ['BX_GSE', 'BY_GSE', 'BZ_GSE', 'BY_GSM', 'BZ_GSM', 'Scalar_B', 
        'Vx', 'Vy', 'Vz', 'Proton_Density', 'Temperature']
key = keys[10]

In [4]:
# Addresses specific

df = pd.read_csv('sc_merge_min_6WdR366jgc_KP2.lst', sep="\s+")

df = df.replace({'Scalar_B':{9999.99:np.nan}, 'Flow_speed':{99999.9:np.nan},
                   'Proton_Density':{999.99:np.nan}, 'Temperature':{9999999.0:np.nan},
                'BX_GSE':{9999.99:np.nan}, 'BY_GSE':{9999.99:np.nan}, 'BZ_GSE':{9999.99:np.nan}, 
                 'BY_GSM':{9999.99:np.nan}, 'BZ_GSM':{9999.99:np.nan}, 'Vx':{99999.9:np.nan}, 
                 'Vy':{99999.9:np.nan}, 'Vz':{99999.9:np.nan}})

print(df.head(25))
df.shape

ts_dict = {}

# Scalar_B-like series
ts_dict['BX_GSE'] = df['BX_GSE'].to_numpy()
ts_dict['BY_GSE'] = df['BY_GSE'].to_numpy()
ts_dict['BZ_GSE'] = df['BZ_GSE'].to_numpy()
ts_dict['BY_GSM'] = df['BY_GSM'].to_numpy()
ts_dict['BZ_GSM'] = df['BZ_GSM'].to_numpy()
ts_dict['Scalar_B'] = df['Scalar_B'].to_numpy()
ts_dict['Vx'] = df['Vx'].to_numpy()
ts_dict['Vy'] = df['Vy'].to_numpy()
ts_dict['Vz'] = df['Vz'].to_numpy()
ts_dict['Proton_Density'] = df['Proton_Density'].to_numpy()
ts_dict['Temperature'] = df['Temperature'].to_numpy()

ts = ts_dict[key]

    Year  Day  Hour  Minute  Scalar_B  BX_GSE  BY_GSE  BZ_GSE  BY_GSM  BZ_GSM  \
0   2014    1     0       0      5.03   -1.07    4.28   -2.39    4.75   -1.22   
1   2014    1     0       1      5.03   -1.00    4.31   -2.39    4.78   -1.21   
2   2014    1     0       2      5.02   -1.11    4.29   -2.35    4.75   -1.18   
3   2014    1     0       3      5.06   -1.15    4.14   -2.66    4.68   -1.52   
4   2014    1     0       4      5.10   -0.98    4.27   -2.60    4.79   -1.43   
5   2014    1     0       5      4.99   -1.05    3.98   -2.81    4.56   -1.70   
6   2014    1     0       6      5.02   -0.95    3.76   -3.17    4.44   -2.11   
7   2014    1     0       7      5.02   -1.05    3.42   -3.51    4.20   -2.53   
8   2014    1     0       8      5.05   -1.02    3.25   -3.70    4.08   -2.75   
9   2014    1     0       9      4.86   -0.45    4.55   -1.46    4.77   -0.26   
10  2014    1     0      10      4.94   -0.26    4.77   -1.16    4.91    0.09   
11  2014    1     0      11 

In [13]:
with open(f"bin/uninterrupted_{key}.pkl", 'rb') as pkl:
    read_res = pickle.load(pkl)

In [14]:
# Interpolation core functions

def MAI(ts, addresses, func, depth=50, mode='center'):
    '''
    depth should be no less than (maximum length of adresses) * 2 + 1
    '''
    arr_res = np.array(ts, copy=True)
    for diapason in addresses:
        for address in range(diapason[0], diapason[1]):
            arr_res[address] = func(ts, address, depth)
    return arr_res       

def meanval_center(ts, index, depth=50):
    length = len(ts)
    index_l = index - np.min([index, depth//2])
    index_h = index + np.min([length-index, depth//2])
    return np.nanmean(ts[index_l: index_h])

def get_addresses(ts, smallest=1, biggest=50):
    idxs_missing = np.argwhere(np.isnan(ts) == False).T[0]
    idx_diff_missing = idxs_missing[1:] - idxs_missing[:-1]
    idx_diff_missing = idx_diff_missing - 1
    idx_diff_missing_nz = idx_diff_missing[np.nonzero(idx_diff_missing)]
    arr_index = np.argwhere((idx_diff_missing <= biggest) * (idx_diff_missing >= smallest) != 0).T[0]
    addresses = []

    for i in arr_index:
        start = idxs_missing[i] + 1
        stop = idxs_missing[i+1]
        addresses.append([start, stop])
    addresses = np.array(addresses)
    return addresses

def get_uninterrupted_addr(ts, smallest=1, biggest=500000):
    idxs_missing = np.argwhere(np.isnan(ts) == True).T[0]
    idx_diff_missing = idxs_missing[1:] - idxs_missing[:-1]
    idx_diff_missing = idx_diff_missing - 1
    idx_diff_missing_nz = idx_diff_missing[np.nonzero(idx_diff_missing)]
    arr_index = np.argwhere((idx_diff_missing <= biggest) * (idx_diff_missing >= smallest) != 0).T[0]
    addresses = []

    for i in arr_index:
        start = idxs_missing[i] + 1
        stop = idxs_missing[i+1]
        addresses.append([start, stop])
    addresses = np.array(addresses)
    return addresses

def get_uninterrupted_data(ts, addresses):
    data = []
    for address in addresses:
        data.append(ts[address[0]:address[1]])
    return data

def make_ecmann_great_again(lyap_list, dim_list):
#     if (np.sum(np.isnan(dim_list)) != 0):
#         return np.nan
    print(len(lyap_list), np.max([np.nanmax(dim_list).astype(int),1]))
    max_dim = np.max([np.nanmax(dim_list).astype(int),1])
    length = len(lyap_list)
    res = np.full((length, max_dim), np.nan)
    for index, item in enumerate(lyap_list):
        if(np.sum(np.isnan(item)) == 0):
            res[index, :item.shape[0]] = item
    return res, max_dim

In [15]:
adrs = get_addresses(ts, smallest=1, biggest=50)
resulting_ts = MAI(ts, adrs, meanval_center, 101)
uninterrupted_addresses = get_uninterrupted_addr(resulting_ts, smallest=1000, biggest=500000)
# uninterrupted_data = get_uninterrupted_data(resulting_ts, uninterrupted_addresses)
# uninterrupted_data

In [16]:
for index, item in enumerate(read_res):
    # Collect all data with right indexing
#     tmp_t = np.arange(uninterrupted_addresses[index, 0], uninterrupted_addresses[index, 1])
    tmp_t = np.array(item['t']) + uninterrupted_addresses[index, 0] + item['window']
    tmp_ts_raw = ts[uninterrupted_addresses[index, 0] + item['window']: uninterrupted_addresses[index, 1]]
    tmp_ts = resulting_ts[uninterrupted_addresses[index, 0] + item['window']: uninterrupted_addresses[index, 1]]
#     tmp_dim = np.full(uninterrupted_addresses[index, 1] - uninterrupted_addresses[index, 0] - item['window'], np.nan)
    tmp_dim = np.array(item['dimension'])
#     tmp_lag = np.full(uninterrupted_addresses[index, 1] - uninterrupted_addresses[index, 0], np.nan)
    tmp_lag = np.array(item['delay'])
#     tmp_lyap_r = np.full(uninterrupted_addresses[index, 1] - uninterrupted_addresses[index, 0], np.nan)
    tmp_lyap_r = np.array(item['rosenstein_lyapunov'])
    # Make Eckmann-Lyapunov great again
    tmp_lyap_e_, md = make_ecmann_great_again(item['eckmann_lyapunov'], item['dimension'])
    tmp_lyap_e = np.full((uninterrupted_addresses[index, 1] - uninterrupted_addresses[index, 0] - item['window'], md), np.nan)
    tmp_lyap_e = tmp_lyap_e_
    # Make column names
    column_names = ['t_min', 'ts_raw', 'ts', 'dim', 'lag', 'lyap_r']
    for i in range(md):
        column_names.append(f"lyap_e_{str(i+1)}")
    # Make final array and dataframe
    arr_r = np.vstack((tmp_t, tmp_ts_raw, tmp_ts, tmp_dim, tmp_lag, tmp_lyap_r))
    arr_e = np.array([col for col in tmp_lyap_e]).T
    arr = np.hstack((arr_r.T, arr_e.T))
    df = pd.DataFrame(arr, columns=column_names)
    # Save csv with right name 
    df.to_csv(f'csv/{key}_#{index}.csv')

3127 6
1904 7
1998 6
871 6
7147 6
31457 7
14090 6
36217 7
147 6
18196 6
8730 7
4391 7
4382 6
23072 7
35185 7
15648 7
1791 6
1728 6
12489 7
1145 6
53233 7
16813 7
26903 7
13809 7
28400 7
11898 6
1902 6
3710 6
12827 7
883 6
6401 7
4040 7
16550 7
7007 7


In [17]:
df

,t_min,ts_raw,ts,dim,lag,lyap_r,lyap_e_1,lyap_e_2,lyap_e_3,lyap_e_4,lyap_e_5,lyap_e_6,lyap_e_7
0,517903.0,56946.0,56946.000000,7.0,5.0,-0.001067,0.085175,0.034873,0.010947,-0.000991,-0.027800,-0.059318,-0.140891
1,517904.0,NaN,70340.292683,5.0,4.0,0.000726,0.150167,0.049052,-0.001869,-0.063241,-0.178451,NaN,NaN
2,517905.0,78408.0,78408.000000,5.0,4.0,0.000051,0.150538,0.050559,-0.001899,-0.064878,-0.177510,NaN,NaN
3,517906.0,78408.0,78408.000000,5.0,4.0,-0.000083,0.149510,0.051041,-0.003100,-0.062107,-0.180008,NaN,NaN
4,517907.0,78408.0,78408.000000,5.0,4.0,0.000302,0.148990,0.049113,-0.000400,-0.062313,-0.180498,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7002,524905.0,162148.0,162148.000000,5.0,5.0,0.001427,0.090136,0.036303,-0.006687,-0.051097,-0.142233,NaN,NaN
7003,524906.0,NaN,178978.188679,6.0,7.0,-0.001064,0.052369,0.024770,0.004706,-0.011753,-0.040258,-0.098837,NaN
7004,524907.0,126907.0,126907.000000,5.0,5.0,0.001979,0.089492,0.034697,-0.003811,-0.049872,-0.142431,NaN,NaN
7005,524908.0,124645.0,124645.000000,6.0,3.0,-0.002346,0.122315,0.055914,0.010985,-0.023379,-0.092459,-0.234608,NaN
